In [1]:
import cobra
import pandas as pd
import numpy as np
import os
import sys

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [3]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/utils.py'>

In [4]:
model = cobra.io.load_json_model('../../../ModelFiles/json/Halo_GEM_v1.json')

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only


In [5]:
utils.test_biomass_production(model)

(<Solution 0.139 at 0x13251ec9e8>, <Solution 16.351 at 0x13251f2550>)

#### 1. Add PHA synthesis reaction
(R)-3-Hydroxybutanoyl-CoA  -> CoA + PHA  
 

|Name                     |MetaCyc ID |
|:------------------------|-----------|
|(R)-3-Hydroxybutanoyl-CoA|CPD-650    |
|CoA                      |CO-A       |

The reactions is catalyzed by phaC gene. It has two isoforms: **TD01GL001170**, **TD01GL001573**. The first gene palys the major rule in PHA synthesis

In [6]:
met_cpd = model.metabolites.get_by_id('CPD-650_c')

In [7]:
for rxn in met_cpd.reactions: print(rxn.id, rxn.reaction,rxn.gene_reaction_rule)

RXN-14255 CPD-15815_c + CPD-650_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c <=> CPD-15815_c + CROTONYL-COA_c TD01GL003540
RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL001906 or TD01GL000448


In [8]:
rxn_pha = cobra.Reaction('PHA_synthetase',name='Poly(3-hydroxyalkanoate) synthetase')
met_pha = cobra.Metabolite('PHA_c',name='PHA',compartment='c')
rxn_pha.add_metabolites({
                            met_cpd                              :-1,
                            model.metabolites.get_by_id('CO-A_c'): 1,
                            met_pha                              : 1,
                        })
rxn_pha.annotation = {'ec-code':'2.3.1-'} # from kegg annotation 
rxn_pha.gene_reaction_rule = 'TD01GL001170 or TD01GL001573'

rxn_pha_sec = cobra.Reaction('PHA_secretion',name='PHA secretion')
rxn_pha_sec.add_metabolites({met_pha:-1})

In [9]:
print(rxn_pha.reaction)
print(rxn_pha_sec.reaction)

CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 


In [10]:
model.add_reactions([rxn_pha,rxn_pha_sec])

In [11]:
utils.set_yeast_extraction(model,lb=0,ub=0)
model.objective='PHA_secretion'
model.objective_direction = 'max'

In [12]:
model.optimize()

,fluxes,reduced_costs
RXN-6021,0.000000,0.000000
DEOXYRIBOSE-P-ALD-RXN,1.500000,0.000000
FEENTERabcpp,0.000000,-0.111111
DHBSZ3FEabcpp,0.000000,0.000000
ABC-3-RXN,0.000000,0.000000
...,...,...
DM_D-ALANINE_p,0.000000,0.000000
NAD-SYNTH-NH3-RXN,0.000000,0.000000
DM_CPD-15815_p,0.000000,0.000000
PHA_synthetase,1.333333,0.000000


In [13]:
## add gene name
utils.correct_genename(model,'TD01GL001170','phaC1')
utils.correct_genename(model,'TD01GL001573','phaC2')

#### 2. Curate gene reaction rules of reactions involved in PHA biosynthesis

##### 2.1 phaB
TD01GL000448 has been experimentally confirmed.

In [14]:
# phaB
utils.print_reactions_of_gene(model,'TD01GL000448')
utils.correct_genename(model,'TD01GL000448','phaB1')

TD01GL000448
(3R)-3-hydroxypentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate --> 3-oxopentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+

TD01GL001906 or TD01GL000448
Acetoacetyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+ --> (3R)-3-hydroxybutanoyl-CoA + Nicotinamide adenine dinucleotide phosphate



##### 2.2 phaA
There are 5 suspected phaA genes, in which TD01GL000367 (phaA3) has been experimentally confirmed. In the model, there are four genes (phaA3,4,5 and TD01GL001328) catalyze this reaction, we keep it as it is. phaA1 and phaA2 in the model catalyze other reactions. A new phaA gene was predicted

In [15]:
# phaA3, only correct gene name for this one
utils.print_reactions_of_gene(model,'TD01GL000367')
utils.correct_genename(model,'TD01GL000367','phaA3')

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A



In [16]:
# phaA4
utils.print_reactions_of_gene(model,'TD01GL001300')

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A



In [17]:
# phaA5
utils.print_reactions_of_gene(model,'TD01GL001318')

TD01GL001318
Coenzyme A + 3-oxopentanoyl-CoA --> Acetyl-CoA + Propanoyl-CoA

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A

TD01GL001318 or TD01GL001328
Acetyl-CoA + Butanoyl-CoA <=> Coenzyme A + 3-Oxohexanoyl-CoA



In [18]:
# phaA1
utils.print_reactions_of_gene(model,'TD01GL003533')

TD01GL003530 and TD01GL003533
Coenzyme A + 3-oxo-5,6-dehydrosuberyl-CoA --> 2,3-dehydroadipyl-CoA + Acetyl-CoA

TD01GL003533
3-Oxoadipyl-CoA + Coenzyme A --> Acetyl-CoA + Succinyl-CoA

TD01GL003533
Coenzyme A + 3-oxo-5,6-dehydrosuberyl-CoA --> Acetyl-CoA + trans-2,3-dehydroadipyl-coA



In [19]:
# phaA2
utils.print_reactions_of_gene(model,'TD01GL000275')

TD01GL000275
Coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxobutanoyl)-CoA --> Acetyl-CoA + (+)-7-epi-jasmonoyl-CoA

TD01GL000275
Coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxohexanoyl)-CoA --> Acetyl-CoA + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-butanoyl-CoA

TD01GL000275
Coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxooctanoyl)-CoA --> Acetyl-CoA + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-hexanoyl-CoA



#### update model files

In [20]:
utils.save_model_into_formats(model,'../../../ModelFiles','Halo_GEM_v1.1')

Saved ../../../ModelFiles/xml/Halo_GEM_v1.1.xml
Saved ../../../ModelFiles/mat/Halo_GEM_v1.1.mat
Saved ../../../ModelFiles/json/Halo_GEM_v1.1.json



#### 4. Add Non-Growth Associated ATP maitanance reaction
1 ATP + 1 H2o --> 1 ADP + 1Pi + 1 H+  
Set the lower bound of the reaction as estimated NGAM, use the value from iML1515 model.

In the model, there will be two reactions for ATP hydrolysis, one with gene reaction rule with a lower bound of 0 and the other without gene reaction rule for NGAM.

In [21]:
halo_model = cobra.io.load_json_model('../../../ModelFiles/json/Halo_GEM_v1.1.json')

In [22]:
utils.test_biomass_production(halo_model)

(<Solution 0.139 at 0x13213a2828>, <Solution 16.351 at 0x13213accf8>)

In [23]:
ngam_rxn = cobra.Reaction('NGAM')
ngam_rxn.add_metabolites({
    halo_model.metabolites.ATP_c  :-1,
    halo_model.metabolites.get_by_id('CPD-15815_c'):-1,
    halo_model.metabolites.ADP_c  :1,
    halo_model.metabolites.Pi_c   : 1,
    halo_model.metabolites.PROTON_c:1,
})

ngam_rxn.lower_bound = 6.86
ngam_rxn.upper_bound = 6.86
ngam_rxn.name = 'Non-Growth Associated ATP maintenance (NGAM)'
print(ngam_rxn)
halo_model.add_reactions([ngam_rxn])
halo_model.repair()

NGAM: ATP_c + CPD-15815_c --> ADP_c + PROTON_c + Pi_c


In [24]:
utils.set_bound(halo_model,'ATPASE-RXN_copy1',lb=0)

In [25]:
utils.test_biomass_production(halo_model)

(<Solution 0.139 at 0x13213c3048>, <Solution 16.401 at 0x13213d27b8>)

In [26]:
utils.save_model_into_formats(halo_model,'../../../ModelFiles','Halo_GEM_v1.2')

Saved ../../../ModelFiles/xml/Halo_GEM_v1.2.xml
Saved ../../../ModelFiles/mat/Halo_GEM_v1.2.mat
Saved ../../../ModelFiles/json/Halo_GEM_v1.2.json



#### 3.2 halo_GEM

#### 4. Batch data

|Name                     |Value |
|:------------------------|-----------|
|Maximal specific growth rate (1/h)|0.52    |
|Glucose uptake rate (mmol/gdDW/h)                     |0.16       |
|P3HB synthesis rate (mmol/gdDW/h)| 0.09|